# A Gentle Introduction to torch.autograd
`torch.autograd` is PyTorch's automatic differentiation engine that powers neural network training.
In this section, you will get a conceptual understanding of how autograd helps a neural network train.

## Background 
Neural networks (NNs) are collection of nested functions that are executed on some input data.
These functions are defined by *parameters* (consisting of weights and biases), which in PyTorch
are stored in tensors.

Training a NN happens in two steps:

**Forward Propagation**: In forward prop, the NN makes its best guess about the correct output. It runs the input data through each of its functions to make this guess.

**Backward Propagation**: In backprop, the NN adjsts its parameters proportionate to the error in its guess. It does this by traversing backwards from the output, collecting the derivatives of the error with respect to the parameters of the functions (gradients), and optimizing the parameters using gradient descent.

## Usage in PyTorch
Let's look at a single training step. For this example, we load a pretrained resnet18 model from `torchvision`. We create a random data tensor to represent a single image with 3 channels, and height & width of 64, and its corresponding `label` initialized to some random values. Label in pretrained models has shape (1,1000).

NOTE: This tutorial work only on CPU and will not work on GPU (even if tensor are moved to CUDA).

In [1]:
import torch, torchvision
model = torchvision.models.resnet18(pretrained=True)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

c:\Users\szymanowskia\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\szymanowskia\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\szymanowskia/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100.0%


Next, we run the input data through the model through each of its layers to make a prediction. 
This is the **forward pass**.

In [2]:
prediction = model(data) # forward pass

We use the model's prediction and the corresponding label to calculate the error (`loss`). The next step is to backpropagate this error through the network. Backward propagation is kicked off when we call `.backward()` on the error tensor. Autograd then calulates and stores the gradients for each model parameter in the parameter's `.grad` attribute. 

In [ ]:
loss = (prediction - labels).sum()
loss.backward() # backward pass

Next, we load an optimizer, in this case SGD with a learning rate of 0.01 and momentum of 0.9. We register all the parameters of the model in the optimizer.

In [4]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

Finally, we call `.step()` to initiate gradient dscent. The optimizer adjusts each parameter by its gradient stored in `.grad`.

In [5]:
optim.step() # gradient descent

## Differentiation in Autograds
Let's take a look at how `autograd` collects gradients. We create two tensors `a` and `b` with `requires_grad=True`.
This signals `autograd` that every operation on them should be tracked.

In [6]:
import torch
a = torch.tensor([2.,3.], requires_grad=True)
b = torch.tensor([6.,4.], requires_grad=True)

We create another tensor `Q` form `a` and `b`
$$
Q = 3a^3 - b^2
$$

In [7]:
Q = 3*a**3 - b**2

Let's assume `a` and `b` to be parameters of an NN, and `Q` to be the error. In NN training, we want gradients of the error w.r.t. parameters, i.e.

$$
    \frac{\partial Q}{\partial a} = 9a^2 \\[10pt]
    \frac{\partial Q}{\partial b} = -2b
$$

When we call `.backward` on `Q`, autograd calculates these gradients and stores them in the respective tensors' `.grad` attribute.

We need to explicitly pass a `gradient` rgument in `Q.backward()` because it is a vector. `gradient` is a tensor of the same shape as `Q`, and it represents the gradient w.r.t. itself, i.e.
$$
    \frac{dQ}{dQ} = 1
$$

Equivalently, we can also aggregate Q into a scalar and call backward implicitly, like `Q.sum().backward()`

In [8]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

Gradients are now deposited in `a.grad` and `b.grad`

In [9]:
# check if collected gradients are correct
print(9*a**2 == a.grad)
print(-2*b == b.grad) 

tensor([True, True])
tensor([True, True])


## Vector Calculus using autograd
Mathematically, if you have a vector valued function $ \vec{y} = f(\vec{x}) $, then gradient of $ \vec{y} $ with respect to $ \vec{x} $ is a jacobian matrix $ J $:
$$
    J = \left( \frac{\partial y}{\partial x_1} \ldots \frac{\partial y}{\partial x_n} \right)^T
    
$$

then by chain rule, the vector-Jacobian product would be the gradient of $ l $ with respect to $ \vec{x} $:

$$
    J^T \cdot \vec{v} =
    \begin{pmatrix}
        \frac{\partial y_1}{\partial x_1} & \cdots & \frac{\partial y_1}{\partial x_n} \\
        \vdots & \ddots & \vdots \\
        \frac{\partial y_m}{\partial x_1} & \cdots & \frac{y_m}{x_n}
    \end{pmatrix}

    \begin{pmatrix}
        \frac{\partial l}{\partial y_1} \\
        \vdots \\
        \frac{\partial l}{\partial y_m}
    \end{pmatrix}
    =
    \begin{pmatrix}
        \frac{\partial l}{\partial x_1} \\
        \vdots \\
        \frac{\partial l}{\partial x_n}
    \end{pmatrix}
$$

This characteristic of vector-Jacobian product is what we use in the above example; `external_grad`
represents $ \vec{v} $

## Computational Graph

Conceptually, autograd keeps a record of data (tensors) & all executed operations (along with the resulting new tensors) in a directed acyclic graph (DAG) consisting of `Function` objects. in this DAG, leaves are the input tensors, roots are the output tensors. By tracing this graph from roots to leaves, you can automatically compute the gradients using the chair rule.

In a forward pass, autograd does two things simultaneously:
- run the requested operation to compute a resulting tensor, and
- maintain the operation's gradient function in the DAG.

The backward pass kick off when `.backward()` is called on the DAG root. `autograd` then:
- computes the gradients from each `.grad_fn`,
- accumulates them in the respective tensor's `.grad` attribute, and
- using the chain rule, propagates all the way to the leaf tensors

NOTE: **DAGs are dynamic in PyToch** An important thing to note is that the graph is recreated from scratch;
after each `.backward()` call, autograd starts populating a new graph. this is exactly what allows you to use control flow statements in your model; you can change the shape, size and operations at every iteration if needed.

## Exclusion from the DAG
`torch.autograd` tracks operations on all tensors which have their `requires_grad` flag set to `True`. For tensors that don't require gradients, setting this attribute to `False` excludes it from the gradient computatio DAG.

The output tensor of an operation will require gradients even if only a single input tensor has 
`requires_grad=True`

In [11]:
x = torch.rand(5, 5)
y = torch.rand(5, 5)
z = torch.rand((5, 5), requires_grad=True)

a = x + y
print(f"Does `a` require gradients? : {a.requires_grad}")
b = x + z
print(f"does `b` require gradients? : {b.requires_grad}")

Does `a` require gradients? : False
does `b` require gradients? : True


In a NN, parameters that don't compute graients are usually called **frozen parameters**. it is useful to "freeze" part of you model if you know in advance that you won't need the gradents of those parameters (this offers some performance benefits by reducing autograd computations).

Another common usecase  where exclusion from the DAG is important is for finetuning a pretrained network.

In finetunig, we freeze most of the models and typically only modify the classifier layers to make predictions on new labels. Let's walk through a small example to demonstrate this. As befor, we load a pretrained resnet18 model, and freeze all the parameters.

In [13]:
from torch import nn, optim
model = torchvision.models.resnet18(pretrained=True)

# Freeze all the parameters in the network
for param in model.parameters():
    param.requires_grad = False

Let's say we want to finetune the model on a new dataset with 10 labels. In resnet, the classifier is the last linear layer `model.fc`. we can simply replace it with a new linear layer (unfrozen by default) that acts as our classifier.

In [15]:
model.fc = nn.Linear(512, 10)

Now all parameters in the model, except the parameters of `model.fc`, are frozen. The only parameters that compute gradients are the weights and bias of `model.fc`

In [16]:
# Optimize only the classifier
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

Notice although we register all the parameters in the optimizer, the only parameters that are computing gradients (and hence updated in gradient descent) are the weights and bias of the classifier.

The same exclusonary functionality is availabe as a context manager in `torch.nograd()`